In [ ]:
from pydantic import BaseModel, Field
import json
from pathlib import Path
from typing import Optional

class APIConfig(BaseModel):
    base_url: str
    timeout: int = 30
    max_retries: int = 3
    retry_delay: float = 1.0

class LLMConfig(BaseModel):
    llm_name: str = "gpt-4o"
    confidence_threshold: float = Field(0.7, ge=0.0, le=1.0)
    temperature: float = Field(0.7, ge=0.0, le=1.0)
    batch_size: int = Field(4, ge=1)

class HeadingsConfig(BaseModel):
    Revenue: List[str] = ["Company-wide revenue", "Segment-wise revenue", "Revenue Drivers"]
    Margins_and_Cashflow: List[str] = ["Profit", "EPS", "Free Cash Flow", "Tax", "Cost Drivers"]
    Guidance: List[str] = ["Playbook", "Guidance", "Projection"]
    Capital_Allocation: List[str] = ["Capital allocation", "Capex", "Share Repurchase"]
    MA: List[str] = ["M&A", "Merger Acquisitions"]
    Inventory_and_Pricing_Strategy: List[str] = ["Inventory", "Pricing Strategy"]
    Macro_Environment: List[str] = ["Macro Environment", "Labor", "Supply Chain"]
    Products_and_RD: List[str] = ["Product Development", "Product Lines", "New Products", "R&D"]

class ECTConfig(BaseModel):
    environment: Environment
    api: APIConfig
    llm: LLMConfig = LLMConfig()
    headings: HeadingsConfig = HeadingsConfig()
    log_level: str = "INFO"

# Environment-specific configs
UAT_CONFIG = {
    "environment": "uat",
    "api": {
        "base_url": "https://vip.imd.qa.search.lex-pii.site.gs.com/searchService/rest/search/bulk/query/table:com.gs.swm.genai",
    }
}

PROD_CONFIG = {
    "environment": "prod",
    "api": {
        "base_url": "https://vip.gsam.prod.search.lex.site.gs.com/searchService/rest/search/bulk/query/table:com.gs.swm.genai",
    }
}

def get_config(env: Optional[str] = None) -> ECTConfig:
    if not env:
        env = os.getenv("ECT_ENVIRONMENT", "prod").lower()
    
    config_data = PROD_CONFIG if env == "prod" else UAT_CONFIG
    return ECTConfig(**config_data)from pydantic import BaseModel, Field
import json
from pathlib import Path
from typing import Optional

class APIConfig(BaseModel):
    base_url: str
    timeout: int = 30
    max_retries: int = 3
    retry_delay: float = 1.0

class LLMConfig(BaseModel):
    llm_name: str = "gpt-4o"
    confidence_threshold: float = Field(0.7, ge=0.0, le=1.0)
    temperature: float = Field(0.7, ge=0.0, le=1.0)
    batch_size: int = Field(4, ge=1)

class HeadingsConfig(BaseModel):
    Revenue: List[str] = ["Company-wide revenue", "Segment-wise revenue", "Revenue Drivers"]
    Margins_and_Cashflow: List[str] = ["Profit", "EPS", "Free Cash Flow", "Tax", "Cost Drivers"]
    Guidance: List[str] = ["Playbook", "Guidance", "Projection"]
    Capital_Allocation: List[str] = ["Capital allocation", "Capex", "Share Repurchase"]
    MA: List[str] = ["M&A", "Merger Acquisitions"]
    Inventory_and_Pricing_Strategy: List[str] = ["Inventory", "Pricing Strategy"]
    Macro_Environment: List[str] = ["Macro Environment", "Labor", "Supply Chain"]
    Products_and_RD: List[str] = ["Product Development", "Product Lines", "New Products", "R&D"]

class ECTConfig(BaseModel):
    environment: Environment
    api: APIConfig
    llm: LLMConfig = LLMConfig()
    headings: HeadingsConfig = HeadingsConfig()
    log_level: str = "INFO"

# Environment-specific configs
UAT_CONFIG = {
    "environment": "uat",
    "api": {
        "base_url": "https://vip.imd.qa.search.lex-pii.site.gs.com/searchService/rest/search/bulk/query/table:com.gs.swm.genai",
    }
}

PROD_CONFIG = {
    "environment": "prod",
    "api": {
        "base_url": "https://vip.gsam.prod.search.lex.site.gs.com/searchService/rest/search/bulk/query/table:com.gs.swm.genai",
    }
}

def get_config(env: Optional[str] = None) -> ECTConfig:
    if not env:
        env = os.getenv("ECT_ENVIRONMENT", "prod").lower()
    
    config_data = PROD_CONFIG if env == "prod" else UAT_CONFIG
    return ECTConfig(**config_data)from pydantic import BaseModel, Field
import json
from pathlib import Path
from typing import Optional

class APIConfig(BaseModel):
    base_url: str
    timeout: int = 30
    max_retries: int = 3
    retry_delay: float = 1.0

class LLMConfig(BaseModel):
    llm_name: str = "gpt-4o"
    confidence_threshold: float = Field(0.7, ge=0.0, le=1.0)
    temperature: float = Field(0.7, ge=0.0, le=1.0)
    batch_size: int = Field(4, ge=1)

class HeadingsConfig(BaseModel):
    Revenue: List[str] = ["Company-wide revenue", "Segment-wise revenue", "Revenue Drivers"]
    Margins_and_Cashflow: List[str] = ["Profit", "EPS", "Free Cash Flow", "Tax", "Cost Drivers"]
    Guidance: List[str] = ["Playbook", "Guidance", "Projection"]
    Capital_Allocation: List[str] = ["Capital allocation", "Capex", "Share Repurchase"]
    MA: List[str] = ["M&A", "Merger Acquisitions"]
    Inventory_and_Pricing_Strategy: List[str] = ["Inventory", "Pricing Strategy"]
    Macro_Environment: List[str] = ["Macro Environment", "Labor", "Supply Chain"]
    Products_and_RD: List[str] = ["Product Development", "Product Lines", "New Products", "R&D"]

class ECTConfig(BaseModel):
    environment: Environment
    api: APIConfig
    llm: LLMConfig = LLMConfig()
    headings: HeadingsConfig = HeadingsConfig()
    log_level: str = "INFO"

# Environment-specific configs
UAT_CONFIG = {
    "environment": "uat",
    "api": {
        "base_url": "https://vip.imd.qa.search.lex-pii.site.gs.com/searchService/rest/search/bulk/query/table:com.gs.swm.genai",
    }
}

PROD_CONFIG = {
    "environment": "prod",
    "api": {
        "base_url": "https://vip.gsam.prod.search.lex.site.gs.com/searchService/rest/search/bulk/query/table:com.gs.swm.genai",
    }
}

def get_config(env: Optional[str] = None) -> ECTConfig:
    if not env:
        env = os.getenv("ECT_ENVIRONMENT", "prod").lower()
    
    config_data = PROD_CONFIG if env == "prod" else UAT_CONFIG
    return ECTConfig(**config_data)